# Quantum State Preparation with Universal Gate Decompositions
This paper [arXiv:1003.5760](https://arxiv.org/abs/1003.5760) by Martin Plesch and Caslav Brukner introduces a state preparation scheme with a $\rm CNOT$ count of $\frac{23}{24}2^n$ in the leading order for even number of qubits. The depth of such scheme is at most half of the number of $\rm CNOT$ gates in the circuit, beating previously known results.

## Parameters
The paper starts by stating that a general pure state of $n$ qubits is described by $2^{n+1} - 2$ real parameters. It also states that on $n$ qubits and $k$ $\rm CNOT$ gates, we are able to introduced up to $4k + 2n$ real parameters. Therefore, we can get a lower bound for the number of $\rm CNOT$ gates in preparing a general $n$-qubit state: $k = \frac{1}{2}2^n$ in the leading order.

## Even number of qubits
First, the decomposition is given for states with even number of qubits, as this is easier and then can be used as a building block for the odd case. The following function gives us the Schmidt decomposition for a state with even number of qubits, which is of main interest for the circuit construction presented in this paper.

In [1]:
import numpy as np
from qiskit.quantum_info import state_fidelity

In [2]:
def schmidt_decomp_even(state):
    n   = int(np.log2(len(state)))
    mat = state.reshape(2 ** (n // 2), 2 ** (n // 2))
    
    u, s, v = np.linalg.svd(mat)
    
    a = [u[:, i].reshape(2 ** (n // 2), 1) for i in range(2 ** (n // 2))]
    b = [v[i, :].reshape(2 ** (n // 2), 1) for i in range(2 ** (n // 2))]    
    
    new_state = np.sum([np.kron(a[i], b[i]) * s[i] for i in range(2 ** (n // 2))], axis=0)    
    assert np.isclose(state_fidelity(state, new_state), 1), "Didnt work :("
    
    return a, b, s

In [3]:
n       = 2 # must be even
state   = np.random.rand(2 ** n)
state   = state / np.linalg.norm(state)
a, b, s = schmidt_decomp_even(state)

This decomposition divides our $2k$-qubit state into the following form:

$$
|\Psi\rangle = \sum_{i=1}^{2k} \alpha_i |\psi\rangle_i |\phi\rangle_i
$$

From the function above, `a` is a list with all $|\psi\rangle_i$, `b` stores $|\phi\rangle_i$, and `s` stores the coefficients $\alpha_i.$ 

Here we will continue with the two-qubit example to keep things simple at first. First, we need to prepare the following state:

$$
\left( \sum_{i=1}^{2^k}\alpha_i|i\rangle \right)|0\rangle
$$

In [4]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector, random_statevector
from sympy import Matrix

In [5]:
qc = QuantumCircuit(n)
qc.ry(2 * np.arccos(s[0]), 0)
qc.draw()

┌─────────────┐
q_0: ┤ RY(0.16854) ├
     └─────────────┘
q_1: ───────────────

In [6]:
Matrix(np.round(Statevector(qc), 5))

Matrix([
[0.99645],
[0.08417],
[      0],
[      0]])

Then, we apply a $\rm CNOT$ gate to get the state $ \sum_{i=1}^{2^k}\alpha_i|i\rangle |i\rangle$.

In [7]:
qc.cx(0, 1)
qc.draw()

┌─────────────┐     
q_0: ┤ RY(0.16854) ├──■──
     └─────────────┘┌─┴─┐
q_1: ───────────────┤ X ├
                    └───┘

In [8]:
Matrix(np.round(Statevector(qc), 5))

Matrix([
[0.99645],
[      0],
[      0],
[0.08417]])

Now, we apply a transformation such that our state is $\sum_{i=1}^{2^k}\alpha_i|\psi_i\rangle |i\rangle$.

In [9]:
theta = 2 * np.arccos(a[0][0][0])
phi   = np.real(np.log(a[0][1][0] / np.sin(np.arccos(a[0][0][0])), dtype='complex') / 1j)
lamb  = np.real(np.log(-a[1][0][0] / np.sin(np.arccos(a[0][0][0])), dtype='complex') / 1j)

In [10]:
qc.u(theta, phi, lamb, 0)
qc.draw()

┌─────────────┐     ┌───────────────┐
q_0: ┤ RY(0.16854) ├──■──┤ U(4.4543,π,0) ├
     └─────────────┘┌─┴─┐└───────────────┘
q_1: ───────────────┤ X ├─────────────────
                    └───┘

In [11]:
Matrix(np.round(Statevector(qc), 5))

Matrix([
[-0.60807],
[-0.78941],
[-0.06668],
[ 0.05136]])

And we do the same for the second qubit so we end up with $\sum_{i=1}^{2^k}\alpha_i|\psi_i\rangle |\phi_i\rangle$.

In [12]:
theta = 2 * np.arccos(b[0][0][0])
phi   = np.real(np.log(b[0][1][0] / np.sin(np.arccos(b[0][0][0])), dtype='complex') / 1j)
lamb  = np.real(np.log(-b[1][0][0] / np.sin(np.arccos(b[0][0][0])), dtype='complex') / 1j)

In [13]:
qc.u(theta, phi, lamb, 1)
qc.draw()

┌─────────────┐     ┌───────────────┐
q_0: ┤ RY(0.16854) ├──■──┤ U(4.4543,π,0) ├
     └─────────────┘┌─┴─┐├───────────────┤
q_1: ───────────────┤ X ├┤ U(5.4637,π,0) ├
                    └───┘└───────────────┘

Finally, we can verify that the final state is the same as the one we generated randomly at the start. 

**State generated with circuit**

In [14]:
Matrix(np.round(Statevector(qc.reverse_bits()), 5))

Matrix([
[ 0.5843],
[0.18108],
[ 0.7036],
[0.36159]])

**Randomly generated state**

In [15]:
Matrix(np.round(state, 5))

Matrix([
[ 0.5843],
[0.18108],
[ 0.7036],
[0.36159]])

As you can see, we obtained exactly what we were looking for! (Except for the fact that we had to use `qc.reverse_bits()`, but that's just because of the discrepancy between the endian used in the paper and in Qiskit.) Thus, we can put this two-qubit state generator in a single function.

In [23]:
def two_qubit_state(state):
    qc = QuantumCircuit(2)
    
    # Normalize state and get Schmidt decomposition
    state   = state / np.linalg.norm(state)
    a, b, s = schmidt_decomp_even(state)

    # Phase 1
    qc.ry(2 * np.arccos(s[0]), 0)
    
    # Phase 2
    qc.cx(0, 1)
    
    # Phase 3
    qc.unitary(np.block([a[0].flatten().T, a[1].flatten().T]).reshape(2, 2).T, 0)
    
    # Phase 4
    qc.unitary(np.block([b[0].flatten().T, b[1].flatten().T]).reshape(2, 2).T, 1)
    
    return qc

In [24]:
state = random_statevector(4).data
qc    = two_qubit_state(state)
qc.decompose().draw()

global phase: -4.7278
     ┌───────────────┐     ┌───────────────────────────┐
q_0: ┤ R(1.0098,π/2) ├──■──┤ U3(1.7278,4.4604,-1.1656) ├
     └───────────────┘┌─┴─┐└──┬─────────────────────┬──┘
q_1: ─────────────────┤ X ├───┤ U3(2.1845,3.3378,0) ├───
                      └───┘   └─────────────────────┘

**State generated with circuit**

In [25]:
circ = Statevector(qc.reverse_bits()).data
Matrix(np.round(circ, 5))

Matrix([
[  0.30604 + 0.3858*I],
[ 0.23186 - 0.40268*I],
[ 0.25725 + 0.19526*I],
[-0.63667 + 0.17869*I]])

**Randomly generated state**

In [26]:
Matrix(np.round(state, 5))

Matrix([
[  0.30604 + 0.3858*I],
[ 0.23186 - 0.40268*I],
[ 0.25725 + 0.19526*I],
[-0.63667 + 0.17869*I]])

As it is evident, we were not considering complex statevectors in the simple walkthrough before. But considering them is not too hard as you can see from the function above. Instead of calculating the angles for a $U$ gate, we can just construct the unitary matrix from the statevectors $|\psi_i\rangle$ and $|\phi_i\rangle$. Qiskit then takes care of converting that into a $U$ gate and keeps track of global phase. To make sure this works, we can run it for a large amount of times and check the state generated is equal to the expected state. 

In [27]:
for _ in range(100):
    state = random_statevector(4).data
    qc    = two_qubit_state(state)
    circ  = Statevector(qc.reverse_bits()).data
        
    assert np.allclose(circ, state)